In [1]:
from scipy.interpolate import interpn
from constant import * 
from multiprocessing import Pool
from functools import partial
import warnings
warnings.filterwarnings("ignore")
np.printoptions(precision=2)

### The value of renting
Assuming we obtain the value: $\tilde{V}_{t+1}(x_{t+1})$ where:   
$x_{t+1} = [w_{t+1}, n_{t+1}, M_{t+1}, g_{t+1} = 0, e_{t+1}, s_{t+1}, z_{t+1}, (H)]$  from interpolation. We know $H$ and $M_t$ from the action taken and we could calculate mortgage payment $m$ and $rh$ (now treated as constant) is observed from the market. 

* Housing choice is limited: $H_{\text{choice}} = \{750, 1000, 1500, 2000\}$
* Mortgage choice is also limitted to discrete values $M_{t} = [0.2H, 0.5H, 0.8H]$ 
* State: continue to rent: $x = [w, n, e, s, z]$ switch to owning a house: $x = [w,n,M,g=0,e,s,z]$ 
* Action: continue to rent: $a = (c, b, k, h)$ switch to owning a house: $a = (c, b, k, M, H)$
* Buying house activities can only happend during the age of 20 and age of 45.

In [2]:
#Define the utility function
def u(c):
    # shift utility function to the left, so it only takes positive value
    return (np.float_power(c, 1-gamma) - 1)/(1 - gamma)

#Define the bequeath function, which is a function of wealth
def uB(tb):
    return B*u(tb)

#Calculate TB_rent
def calTB_rent(x):
    # change input x as numpy array
    # w, n, e, s, z = x
    TB = x[:,0] + x[:,1]
    return TB

#Calculate TB_own 
def calTB_own(x):
    # change input x as numpy array
    # transiton from (w, n, e, s, z) -> (w, n, M, 0, e, s, z, H)
    TB = x[:,0] + x[:,1] + x[:,7]*pt - x[:,2]
    return TB

#Reward function for renting
def u_rent(a):
    '''
    Input:
        action a: c, b, k, h = a 
    Output: 
        reward value: the length of return should be equal to the length of a
    '''
    c = a[:,0]
    h = a[:,3]
    C = np.float_power(c, alpha) * np.float_power(h, 1-alpha)
    return u(C)

#Reward function for owning 
def u_own(a):
    '''
    Input:
        action a: c, b, k, M, H = a
    Output: 
        reward value: the length of return should be equal to the length of a
    '''
    c = a[:,0]
    H = a[:,4]
    C = np.float_power(c, alpha) * np.float_power((1+kappa)*H, 1-alpha)
    return u(C)

#Define the earning function, which applies for both employment and unemployment, good econ state and bad econ state 
def y(t, x):
    w, n, e, s, z = x
    if t <= T_R:
        return detEarning[t] * earningShock[int(s)] * e + (1-e) * welfare
    else:
        return detEarning[t]
    
#Earning after tax and fixed by transaction in and out from 401k account 
def yAT(t,x):
    yt = y(t, x)
    w, n, e, s, z = x
    if t <= T_R and e == 1:
        # 5% of the income will be put into the 401k 
        return (1-tau_L)*(yt * (1-yi))
    if t <= T_R and e == 0:
        return yt
    else:
        # t > T_R, n/discounting amount will be withdraw from the 401k 
        return (1-tau_R)*yt + n/Dt[t]

#Define the evolution of the amount in 401k account 
def gn(t, n, x, s_next):
    w, n, e, s, z = x
    if t <= T_R and e == 1:
        # if the person is employed, then 5 percent of his income goes into 401k 
        n_cur = n + y(t, x) * yi
    elif t <= T_R and e == 0:
        # if the perons is unemployed, then n does not change 
        n_cur = n
    else:
        # t > T_R, n/discounting amount will be withdraw from the 401k 
        n_cur = n - n/Dt[t]
    return (1+r_k[int(s), s_next])*n_cur 

In [3]:
def transition_to_rent(x,a,t):
    '''
        imput: a is np array constains all possible actions 
        output: from x = [w, n, e, s, z] to x = [w, n, e, s, z]
    '''
    w, n, e, s, z = x
    # variables used to collect possible states and probabilities
    x_next = []
    prob_next = []
    for aa in a:
        c, b, k, h = aa
        # transition of z
        if z == 1:
            z_next = 1
        else:
            if k == 0:
                z_next = 0
            else:
                z_next = 1 
                
        for s_next in [0,1]:
            w_next =  b*(1+r_b[int(s)]) + k*(1+r_k[int(s), s_next])
            n_next = gn(t, n, x, s_next)
            if t >= T_R:
                e_next = 0
                x_next.append([w_next, n_next, e_next, s_next, z_next])
                prob_next.append(Ps[int(s),s_next])
            else:
                for e_next in [0,1]:
                    x_next.append([w_next, n_next, e_next, s_next, z_next])
                    prob_next.append(Ps[int(s),s_next] * Pe[int(s),s_next,int(e),e_next])
    return np.array(x_next), np.array(prob_next)    
  

def transition_to_own(x,a,t):
    '''
        imput a is np array constains all possible actions 
        from x = [w, n, e, s] to x = [w, n, M, g=0, e, s, H]
    '''
    w, n, e, s = x
    # variables used to collect possible states and probabilities
    x_next = []
    prob_next = []
    for aa in a:
        c, b, k, M, H = aa
        M_next = M*(1+rh)
        # transition of z
        if z == 1:
            z_next = 1
        else:
            if k == 0:
                z_next = 0
            else:
                z_next = 1 
                
        for s_next in [0,1]:
            w_next =  b*(1+r_b[int(s)]) + k*(1+r_k[int(s), s_next])
            n_next = gn(t, n, x, s_next)
            if t >= T_R:
                e_next = 0
                x_next.append([w_next, n_next, M_next, 0, e_next, s_next, z_next, H])
                prob_next.append(Ps[int(s),s_next])
            else:
                for e_next in [0,1]:
                    x_next.append([w_next, n_next, M_next, 0, e_next, s_next, z_next, H])
                    prob_next.append(Ps[int(s),s_next] * Pe[int(s),s_next,int(e),e_next])
    return np.array(x_next), np.array(prob_next)

In [7]:
class Approxy(object):
    def __init__(self, pointsRent, Vrent, Vown, t):
        self.Vrent = Vrent 
        self.Vown = Vown
        self.Prent = pointsRent
        self.t = t
    def predict(self, xx):
        if xx.shape[1] == 5:
            # x = [w, n, e, s, z]
            pvalues = np.zeros(xx.shape[0])
            for e in [0,1]:
                for s in [0,1]:
                    for z in [0,1]: 
                        index = (xx[:,2] == e) & (xx[:,3] == s) & (xx[:,4] == z)
                        pvalues[index]=interpn(self.Prent, self.Vrent[:,:,e,s,z], xx[index00][:,:2], 
                                               bounds_error = False, fill_value = None)
            return pvalues
        else: 
            # x = w, n, M, g=0, e, s, z, H
            pvalues = np.zeros(xx.shape[0])
            for i in range(len(H_options)):
                H = H_options[i]
                # Mortgage amount, * 0.25 is the housing price per unit
                Ms = np.array([0.01*H,0.05*H,0.1*H,0.2*H,0.3*H,0.4*H,0.5*H,0.6*H,0.7*H,0.8*H]) * pt
                points = (ws,ns,Ms)
                for e in [0,1]:
                    for s in [0,1]:
                        for z in [0,1]: 
                            index = (xx[:,4] == e) & (xx[:,5] == s) & (xx[:,6] == z) & (xx[:,7] == H)
                            pvalues[index]=interpn(points, self.Vown[i][:,:,:,0,e,s,z,self.t], xx[index00][:,:3], 
                                                   method = "nearest",bounds_error = False, fill_value = None)
            return pvalues

# used to calculate dot product
def dotProduct(p_next, uBTB, t):
    if t >= T_R:
        return (p_next*uBTB).reshape((len(p_next)//2,2)).sum(axis = 1)
    else:
        return (p_next*uBTB).reshape((len(p_next)//4,4)).sum(axis = 1)
    
# Value function is a function of state and time, according to the restriction transfer from renting to ownning can only happen
# between the age: 0 - 25
def V(x, t, NN):
    w, n, e, s,z = x
    yat = yAT(t,x)
    
    # first define the objective function solver and then the objective function
    def obj_solver_rent(obj_rent):
        # a = [c, b, k, h] 
        # Constrain: yat + w = c + b + k + pr*h
        actions = []
        for hp in np.linspace(0.001,0.999,20):
            budget1 = yat + w
            h = budget1 * hp/pr
            budget2 = budget1 * (1-hp)
            for cp in np.linspace(0.001,0.999,11):
                c = budget2*cp
                budget3 = budget2 * (1-cp)
                #.....................stock participation cost...............
                for kp in np.linspace(0,1,11):
                    # If z == 1 pay for matainance cost Km = 0.5
                    if z == 1:
                        # kk is stock allocation
                        kk = budget3 * kp
                        if kk > Km:
                            k = kk - Km
                            b = budget3 * (1-kp)
                        else:
                            k = 0
                            b = budget3
                    # If z == 0 and k > 0 payfor participation fee Kc = 5
                    else:
                        kk = budget3 * kp 
                        if kk > Kc:
                            k = kk - Kc
                            b = budget3 * (1-kp)
                        else:
                            k = 0
                            b = budget3
                #..............................................................
         
        actions = np.array(actions)
        values = obj_rent(actions)
        fun = np.max(values)
        ma = actions[np.argmax(values)]
        return fun, ma          
                    
    def obj_solver_own(obj_own):
    # a = [c, b, k, M, H]
    # possible value of H = {750, 1000, 1500, 2000} possible value of [0.2H, 0.5H, 0.8H]]*pt
    # (M, t, rh) --> m 
    # Constrain: yat + w = c + b + k + (H*pt - M) + ch
        actions = []
        for H in H_options:
            for mp in M_options:
                M = mp*H*pt
                m = M/D[T_max - t]
                # 5 is the welfare income which is also the minimum income
                if (H*pt - M) + c_h <= yat + w and m < pr*H + 5:
                    budget1 = yat + w - (H*pt - M) - c_h
                    for cp in np.linspace(0.001,0.999,11):
                        c = budget1*cp
                        budget2 = budget1 * (1-cp)
                        #.....................stock participation cost...............
                        for kp in np.linspace(0,1,11):
                            # If z == 1 pay for matainance cost Km = 0.5
                            if z == 1:
                                # kk is stock allocation
                                kk = budget2 * kp
                                if kk > Km:
                                    k = kk - Km
                                    b = budget2 * (1-kp)
                                else:
                                    k = 0
                                    b = budget2
                            # If z == 0 and k > 0 payfor participation fee Kc = 5
                            else:
                                kk = budget2 * kp 
                                if kk > Kc:
                                    k = kk - Kc
                                    b = budget2 * (1-kp)
                                else:
                                    k = 0
                                    b = budget2
                        #..............................................................            
                            
        if len(actions) == 0:
            return -np.inf, [0,0,0,0,0]
        else:
            actions = np.array(actions)
            values = obj_own(actions)
            fun = np.max(values)
            ma = actions[np.argmax(values)]
            return fun, ma
    
    if t == T_max-1:
        # The objective function of renting
        def obj_rent(actions): 
            # a = [c, b, k, h]
            x_next, p_next  = transition_to_rent(x, actions, t)
            uBTB = uB(calTB_rent(x_next)) 
            return u_rent(actions) + beta * dotProduct(uBTB, p_next, t) 

        fun, action = obj_solver_rent(obj_rent)
        return np.array([fun, action])
    
    # If the agent is older that 45 or if the agent is unemployed then keep renting 
    elif t > 25 or e == 0:
        # The objective function of renting
        def obj_rent(actions):
            # a = [c, b, k, h]
            x_next, p_next  = transition_to_rent(x, actions, t)
            V_tilda = NN.predict(x_next) # V_rent_{t+1} used to approximate, shape of x is [w,n,e,s]
            uBTB = uB(calTB_rent(x_next))
            return u_rent(actions) + beta * (Pa[t] * dotProduct(V_tilda, p_next, t) + (1 - Pa[t]) * dotProduct(uBTB, p_next, t))

        fun, action = obj_solver_rent(obj_rent)
        return np.array([fun, action])
    # If the agent is younger that 45 and agent is employed. 
    else:
        # The objective function of renting
        def obj_rent(actions):
            # a = [c, b, k, h]
            x_next, p_next  = transition_to_rent(x, actions, t)
            V_tilda = NN.predict(x_next) # V_rent_{t+1} used to approximate, shape of x is [w,n,e,s]
            uBTB = uB(calTB_rent(x_next))
            return u_rent(actions) + beta * (Pa[t] * dotProduct(V_tilda, p_next, t) + (1 - Pa[t]) * dotProduct(uBTB, p_next, t))
        # The objective function of owning
        def obj_own(actions):
            # a = [c, b, k, M, H]
            x_next, p_next  = transition_to_own(x, actions, t)
            V_tilda = NN.predict(x_next) # V_own_{t+1} used to approximate, shape of x is [w, n, M, g=0, e, s, H]
            uBTB = uB(calTB_own(x_next))
            return u_own(actions) + beta * (Pa[t] * dotProduct(V_tilda, p_next, t) + (1 - Pa[t]) * dotProduct(uBTB, p_next, t))

        fun1, action1 = obj_solver_rent(obj_rent)
        fun2, action2 = obj_solver_own(obj_own)
        if fun1 > fun2:
            return np.array([fun1, action1])
        else:
            return np.array([fun2, action2])

In [9]:
# wealth discretization 
ws = np.array([10,25,50,75,100,125,150,175,200,250,500,750,1000,1500,3000])
w_grid_size = len(ws)
# 401k amount discretization 
ns = np.array([1, 5, 10, 15, 25, 40, 65, 100, 150, 300, 400,1000])
n_grid_size = len(ns)
pointsRent = (ws, ns)
# dimentions of the state
dim = (w_grid_size, n_grid_size,2,2,2)
dimSize = len(dim)

xgrid = np.array([[w, n, e, s, z]
                            for w in ws
                            for n in ns
                            for e in [0,1]
                            for s in [0,1]
                            for z in [0,1]
                            ]).reshape(dim + (dimSize,))

xs = xgrid.reshape((np.prod(dim),dimSize))

Vgrid = np.zeros(dim + (T_max,))
cgrid = np.zeros(dim + (T_max,))
bgrid = np.zeros(dim + (T_max,))
kgrid = np.zeros(dim + (T_max,))
hgrid = np.zeros(dim + (T_max,))
# Policy function of buying a house 
Mgrid = np.zeros(dim + (T_max,))
Hgrid = np.zeros(dim + (T_max,))

# # Define housing choice part: Housing unit options and Mortgage amount options
V1000 = np.load("Vgrid1000.npy")
V1500 = np.load("Vgrid1500.npy")
V2000 = np.load("Vgrid2000.npy")
V750 = np.load("Vgrid750.npy")
H_options = [750, 1000, 1500, 2000]
M_options = [0.2, 0.5, 0.8]
Vown = [V750, V1000, V1500, V2000]

In [11]:
%%time
# value iteration part 
pool = Pool()
for t in range(T_max-1,T_min, -1):
    print(t)
    if t == T_max - 1:
        f = partial(V, t = t, NN = None)
        results = np.array(pool.map(f, xs))
    else:
        approx = Approxy(pointsRent,Vgrid[:,:,:,:,:,t+1], Vown, t+1)
        f = partial(V, t = t, NN = approx)
        results = np.array(pool.map(f, xs))
    # a = [c,b,k,h] or a = [c,b,k,M,H]
    Vgrid[:,:,:,:,:,t] = results[:,0].reshape(dim)
    cgrid[:,:,:,:,:,t] = np.array([r[0] for r in results[:,1]]).reshape(dim)
    bgrid[:,:,:,:,:,t] = np.array([r[1] for r in results[:,1]]).reshape(dim)
    kgrid[:,:,:,:,:,t] = np.array([r[2] for r in results[:,1]]).reshape(dim)
    # if a = [c, b, k, h]
    hgrid[:,:,:,:,:t] = np.array([r[3] if len(r) == 4 else r[4] for r in results[:,1]]).reshape(dim)
    # if a = [c, b, k, M, H]
    Mgrid[:,:,:,:,:,t] = np.array([r[3] if len(r) == 5 else 0 for r in results[:,1]]).reshape(dim)
    Hgrid[:,:,:,:,:,t] = np.array([r[4] if len(r) == 5 else 0 for r in results[:,1]]).reshape(dim)
pool.close()

np.save("Vgrid_renting",Vgrid) 
np.save("cgrid_renting",cgrid) 
np.save("bgrid_renting",bgrid) 
np.save("kgrid_renting",kgrid) 
np.save("hgrid_renting",hgrid) 
np.save("Mgrid_renting",Mgrid) 
np.save("Hgrid_renting",Hgrid) 

In [10]:
for tt in range(1,25):
    print(Hgrid[:,1,1,1,tt])

[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.  750. 1000.    0. 1500. 1500. 2000. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.    0. 1000.    0. 1500. 1500. 2000. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.  750.    0. 1000.    0. 1500. 1500. 2000. 2000. 2000. 200

In [11]:
for tt in range(1,25):
    print(Hgrid[:,1,0,1,tt])

[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
    0. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
    0. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
    0. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
    0. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 200

In [12]:
for tt in range(1,25):
    print(Hgrid[:,1,1,0,tt])

[   0.    0.    0.  750.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750.  750. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750.  750. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750.  750. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.  750.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.  750.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.  750.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.  750.  750. 1000. 1000. 1500. 1500. 1500. 2000. 2000. 200

In [13]:
for tt in range(1,25):
    print(Hgrid[:,1,0,0,tt])

[   0.    0.    0.  750.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.  750.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750. 1000. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 2000. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 2000.
 2000. 1500. 2000.]
[   0.    0.    0.    0.  750.  750. 1000. 1500. 1500. 2000. 2000. 200